In [1]:
import pandas as pd
import sqlite3

# Membuat koneksi ke database SQLite
conn = sqlite3.connect('churn_prediction.db')

In [2]:
# Load dataset ke dalam DataFrame
data = pd.read_csv('https://raw.githubusercontent.com/imomenujung/SQL-Project/main/Data-Analyst/Churn-Prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv', encoding='ISO-8859-1')

# Impor data ke tabel SQL
data.to_sql('telco_churn', conn, if_exists='replace', index=False)

7043

In [3]:
# Checking Null Value
null_values = data.isnull().sum()
print(null_values)


customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64


In [4]:
queries = {
"Count Total Customer" : """
  SELECT COUNT(*) FROM telco_churn;""",
"Count Churned vs. Non-Churned Customers" : '''
  SELECT Churn, COUNT(*) FROM telco_churn GROUP BY Churn;''',
"Average Tenure of Churned vs. Non-Churned Customers:" : '''
  SELECT Churn, AVG(Tenure) FROM telco_churn GROUP BY Churn;''',
"Distribution of Payment Methods among Churned Customers:" : '''
  SELECT PaymentMethod, COUNT(*) FROM telco_churn WHERE Churn = 'Yes' GROUP BY PaymentMethod;'''
}
for query_name, query in queries.items():
    print(f"{query_name}")
    result = pd.read_sql_query(query, conn)
    print(result)
    print("\n")



Count Total Customer
   COUNT(*)
0      7043


Count Churned vs. Non-Churned Customers
  Churn  COUNT(*)
0    No      5174
1   Yes      1869


Average Tenure of Churned vs. Non-Churned Customers:
  Churn  AVG(Tenure)
0    No    37.569965
1   Yes    17.979133


Distribution of Payment Methods among Churned Customers:
               PaymentMethod  COUNT(*)
0  Bank transfer (automatic)       258
1    Credit card (automatic)       232
2           Electronic check      1071
3               Mailed check       308




In [5]:
# Add tenure_group Column
conn.execute("ALTER TABLE telco_churn ADD COLUMN tenure_group VARCHAR(20);")

# Update tenure_group Column
conn.execute('''
UPDATE telco_churn
SET tenure_group = CASE
    WHEN tenure <= 12 THEN '0-12 Months'
    WHEN tenure <= 24 THEN '12-24 Months'
    WHEN tenure <= 48 THEN '24-48 Months'
    WHEN tenure <= 60 THEN '48-60 Months'
    ELSE '60+ Months'
END;
''')

# Commit
conn.commit()

# Changed Verification
result = pd.read_sql_query("SELECT * FROM telco_churn;", conn)
print(result)


      customerID  gender  SeniorCitizen Partner Dependents  tenure  \
0     7590-VHVEG  Female              0     Yes         No       1   
1     5575-GNVDE    Male              0      No         No      34   
2     3668-QPYBK    Male              0      No         No       2   
3     7795-CFOCW    Male              0      No         No      45   
4     9237-HQITU  Female              0      No         No       2   
...          ...     ...            ...     ...        ...     ...   
7038  6840-RESVB    Male              0     Yes        Yes      24   
7039  2234-XADUH  Female              0     Yes        Yes      72   
7040  4801-JZAZL  Female              0     Yes        Yes      11   
7041  8361-LTMKD    Male              1     Yes         No       4   
7042  3186-AJIEK    Male              0      No         No      66   

     PhoneService     MultipleLines InternetService OnlineSecurity  ...  \
0              No  No phone service             DSL             No  ...   
1        

In [6]:
# Add charges_group Column
conn.execute("ALTER TABLE telco_churn ADD COLUMN charges_group VARCHAR(20);")

# Update charges_group Column
conn.execute('''
UPDATE telco_churn
SET charges_group = CASE
    WHEN MonthlyCharges <= 30 THEN 'Low'
    WHEN MonthlyCharges <= 60 THEN 'Medium'
    ELSE 'High'
END;
''')

# Commit
conn.commit()

# Changed Verification
result = pd.read_sql_query("SELECT * FROM telco_churn ;", conn)
print(result)


      customerID  gender  SeniorCitizen Partner Dependents  tenure  \
0     7590-VHVEG  Female              0     Yes         No       1   
1     5575-GNVDE    Male              0      No         No      34   
2     3668-QPYBK    Male              0      No         No       2   
3     7795-CFOCW    Male              0      No         No      45   
4     9237-HQITU  Female              0      No         No       2   
...          ...     ...            ...     ...        ...     ...   
7038  6840-RESVB    Male              0     Yes        Yes      24   
7039  2234-XADUH  Female              0     Yes        Yes      72   
7040  4801-JZAZL  Female              0     Yes        Yes      11   
7041  8361-LTMKD    Male              1     Yes         No       4   
7042  3186-AJIEK    Male              0      No         No      66   

     PhoneService     MultipleLines InternetService OnlineSecurity  ...  \
0              No  No phone service             DSL             No  ...   
1        

In [8]:
# prompt: Make Dataframe from result

import pandas as pd
df = pd.DataFrame(result)
print(df)


      customerID  gender  SeniorCitizen Partner Dependents  tenure  \
0     7590-VHVEG  Female              0     Yes         No       1   
1     5575-GNVDE    Male              0      No         No      34   
2     3668-QPYBK    Male              0      No         No       2   
3     7795-CFOCW    Male              0      No         No      45   
4     9237-HQITU  Female              0      No         No       2   
...          ...     ...            ...     ...        ...     ...   
7038  6840-RESVB    Male              0     Yes        Yes      24   
7039  2234-XADUH  Female              0     Yes        Yes      72   
7040  4801-JZAZL  Female              0     Yes        Yes      11   
7041  8361-LTMKD    Male              1     Yes         No       4   
7042  3186-AJIEK    Male              0      No         No      66   

     PhoneService     MultipleLines InternetService OnlineSecurity  ...  \
0              No  No phone service             DSL             No  ...   
1        

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder

In [10]:
# Memisahkan kolom berdasarkan tipe data
categorical_columns = df.select_dtypes(include=['object']).columns

# Membuat objek LabelEncoder
label_encoder = LabelEncoder()

# Mengaplikasikan LabelEncoder pada kolom-kolom kategorikal
for column in categorical_columns:
    df[column] = label_encoder.fit_transform(df[column])

# Menampilkan data setelah proses encoding
print(df)

      customerID  gender  SeniorCitizen  Partner  Dependents  tenure  \
0           5375       0              0        1           0       1   
1           3962       1              0        0           0      34   
2           2564       1              0        0           0       2   
3           5535       1              0        0           0      45   
4           6511       0              0        0           0       2   
...          ...     ...            ...      ...         ...     ...   
7038        4853       1              0        1           1      24   
7039        1525       0              0        1           1      72   
7040        3367       0              0        1           1      11   
7041        5934       1              1        1           0       4   
7042        2226       1              0        0           0      66   

      PhoneService  MultipleLines  InternetService  OnlineSecurity  ...  \
0                0              1                0          

In [ ]:
# Mengembalikan nilai yang telah di-encode ke nilai aslinya
for column in categorical_columns:
    df[column] = label_encoders[column].inverse_transform(df[column])

# Menampilkan data setelah mengembalikan ke nilai asli
print("\nDecoded Data:")
print(df)

In [11]:
X = df.drop('Churn', axis=1)
y = df['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [14]:
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.7963094393186657
[[940  96]
 [191 182]]
              precision    recall  f1-score   support

           0       0.83      0.91      0.87      1036
           1       0.65      0.49      0.56       373

    accuracy                           0.80      1409
   macro avg       0.74      0.70      0.71      1409
weighted avg       0.78      0.80      0.79      1409



In [16]:
queries = {
"Churn Rate by Tenuure Group" : """
  SELECT tenure_group, Churn, COUNT(*) FROM telco_churn GROUP BY tenure_group, Churn;""",
"Churn Rate by Charges Group" : '''
  SELECT charges_group, Churn, COUNT(*) FROM telco_churn GROUP BY charges_group, Churn;'''
}
for query_name, query in queries.items():
    print(f"{query_name}")
    result = pd.read_sql_query(query, conn)
    print(result)
    print("\n")


Churn Rate by Tenuure Group
   tenure_group Churn  COUNT(*)
0   0-12 Months    No      1149
1   0-12 Months   Yes      1037
2  12-24 Months    No       730
3  12-24 Months   Yes       294
4  24-48 Months    No      1269
5  24-48 Months   Yes       325
6  48-60 Months    No       712
7  48-60 Months   Yes       120
8    60+ Months    No      1314
9    60+ Months   Yes        93


Churn Rate by Charges Group
  charges_group Churn  COUNT(*)
0          High    No      2746
1          High   Yes      1379
2           Low    No      1491
3           Low   Yes       162
4        Medium    No       937
5        Medium   Yes       328


